In [ ]:
from action_embed import ActionEmbedding
import torch

embedder = ActionEmbedding(num_actions=10000, embed_dim=128, normalize=True, dropout=0.1)

action_index = 42
embedded_action = embedder(action_index)  # shape: [128]
print(embedded_action.shape)
print(embedded_action)

batch_actions = torch.tensor([1, 5, 42])
embedded_batch = embedder(batch_actions)  # shape: [3, 128]
print(embedded_batch.shape)
print(embedded_batch)


In [ ]:
import torch
from transition_decoder import DecoderTransformerConfig, DecoderTransformerTorch

# Create dummy config
config = DecoderTransformerConfig(
    emb_dim=32, # embedding dimension
    num_heads=4,
    attention_dropout_rate=0.1,
    dropout_rate=0.1,
    mlp_dim_factor=4,
    activation= 'relu' ,
    use_bias=True,
    max_rows=30,
    max_cols=30,
    vocab_size=10,
    num_layers=2
)

# For testing purposes, assign transformer_layer attribute
config.transformer_layer = config

decoder = DecoderTransformerTorch(config)
decoder.eval()  # set evaluation mode

embedded_action = torch.randn(1,config.emb_dim) 
embedded_state = torch.randn(1,config.emb_dim) 

device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f'Using device:' , {device})

decoder = decoder.to(device)
embedded_action = embedded_action.to(device)
embedded_state = embedded_state.to(device)

# Run forward
shape_row_logits, shape_col_logits, grid_logits = decoder(embedded_action, embedded_state, dropout_eval=True)


Using device: mps for world_model/transformer.py
Using device: {'mps'}
embedded_action shape: torch.Size([5, 32])
embedded_state shape: torch.Size([8, 32])


AssertionError: Both inputs must be 3D tensors (batch, seq_len, emb_dim)

In [4]:
# Expected output shapes
assert shape_row_logits.shape == (B, config.max_rows), f"Expected shape_row_logits {(B, config.max_rows)}, got {shape_row_logits.shape}"
assert shape_col_logits.shape == (B, config.max_cols), f"Expected shape_col_logits {(B, config.max_cols)}, got {shape_col_logits.shape}"
expected_grid_tokens = (T_action + T_state) - (T_action + 3)
assert grid_logits.shape == (B, expected_grid_tokens, config.vocab_size), f"Expected grid_logits {(B, expected_grid_tokens, config.vocab_size)}, got {grid_logits.shape}"

print('All tests passed!')

All tests passed!
